In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from datasets import load_dataset
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import wandb
from wandb.integration.keras import WandbCallback
import os
import json
from datetime import datetime

c:\Users\radit\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("garythung/trashnet")

In [3]:
def preprocess_data(dataset):
    images = []
    labels = []
    
    for item in dataset:
        # Resize and convert to np
        img = tf.image.resize(item['image'], (224, 224))
        img = tf.keras.utils.img_to_array(img)
        images.append(img)
        labels.append(item['label'])
    
    return np.array(images), np.array(labels)

# Preprocess the data
X, y = preprocess_data(dataset['train'])

# Convert labels to categorical
num_classes = len(np.unique(y))
y = to_categorical(y, num_classes)

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
class TrainDataGenerator:
    def __init__(self, generator, x, y, batch_size):
        self.generator = generator
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.steps = len(x) // batch_size
        
    def __iter__(self):
        batch_generator = self.generator.flow(self.x, self.y, batch_size=self.batch_size)
        self.batch_metrics = {'train_loss': [], 'train_accuracy': []}
        
        for i in range(self.steps):
            batch = next(batch_generator)
            yield batch
            
    def on_batch_end(self, batch, logs=None):
        if logs:
            self.batch_metrics['train_loss'].append(logs.get('loss', 0))
            self.batch_metrics['train_accuracy'].append(logs.get('accuracy', 0))
            
    def get_epoch_metrics(self):
        if self.batch_metrics['train_loss']:
            return {
                'train_loss': np.mean(self.batch_metrics['train_loss']),
                'train_accuracy': np.mean(self.batch_metrics['train_accuracy'])
            }

In [5]:
class WandbMetricsLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        wandb.log({
            'epoch': epoch,
            'train_loss': logs.get('loss', 0),
            'train_accuracy': logs.get('accuracy', 0),
            'val_loss': logs.get('val_loss', 0),
            'val_accuracy': logs.get('val_accuracy', 0),
            'learning_rate': float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        })

In [6]:
def create_model(num_classes):
    model = models.Sequential([
        # CNN layers
        layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        # Dense layers
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
BATCH_SIZE = 32

wandb.init(
    project="trashnet",  
    config={
        "learning_rate": 0.001,
        "epochs": 50,
        "batch_size": BATCH_SIZE,
        "architecture": "CNN",
        "optimizer": "adam",
        "loss": "categorical_crossentropy",
        "dataset": "trashnet",
        "num_classes": num_classes,
    },
    name="CPU-CNN-Tensorflow-2"
)

model = create_model(num_classes)
optimizer = tf.keras.optimizers.Adam(learning_rate=wandb.config.learning_rate)
model.compile(
    optimizer=optimizer,
    loss=wandb.config.loss,
    metrics=['accuracy']    
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: raditsoic (soic). Use `wandb login --relogin` to force relogin


c:\Users\radit\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:

train_generator = train_datagen.flow(
    X_train, y_train,
    batch_size=wandb.config.batch_size,
    shuffle=True
)

val_generator = val_datagen.flow(
    X_val, y_val,
    batch_size=wandb.config.batch_size,
    shuffle=False
)

In [10]:
callbacks = [
    WandbMetricsLogger(),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )
]

In [11]:
history = model.fit(
    iter(train_generator),
    epochs=wandb.config.epochs,
    validation_data=val_datagen.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    validation_steps=len(X_val) // BATCH_SIZE,
    callbacks=callbacks
)

c:\Users\radit\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 187s 1s/step - accuracy: 0.3957 - loss: 1.8853 - val_accuracy: 0.2379 - val_loss: 2.1846 - learning_rate: 0.0010
Epoch 2/50
  1/126 ━━━━━━━━━━━━━━━━━━━━ 1:30 722ms/step - accuracy: 0.4062 - loss: 1.7231

c:\Users\radit\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


126/126 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.4062 - loss: 1.7231 - val_accuracy: 0.2631 - val_loss: 2.1568 - learning_rate: 0.0010
Epoch 3/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 111s 880ms/step - accuracy: 0.5417 - loss: 1.2506 - val_accuracy: 0.2188 - val_loss: 6.4187 - learning_rate: 0.0010
Epoch 4/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 6s 41ms/step - accuracy: 0.3750 - loss: 1.3084 - val_accuracy: 0.2177 - val_loss: 6.4450 - learning_rate: 0.0010
Epoch 5/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 114s 905ms/step - accuracy: 0.5634 - loss: 1.1673 - val_accuracy: 0.3024 - val_loss: 2.0319 - learning_rate: 0.0010
Epoch 6/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.4375 - loss: 1.6584 - val_accuracy: 0.3488 - val_loss: 1.8744 - learning_rate: 0.0010
Epoch 7/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 115s 916ms/step - accuracy: 0.5849 - loss: 1.1033 - val_accuracy: 0.3246 - val_loss: 2.0410 - learning_rate: 0.0010
Epoch 8/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - accuracy: 0.6364 - loss: 0.97

In [12]:
def log_model_to_wandb(model, config, metrics, model_version):
    model_name = f"trashnet_model_v{model_version}"
    artifact = wandb.Artifact(
        name=model_name,
        type="model",
        description="Trashnet Classification model",
        metadata=config
    )
    
    # Save model locally first
    model_dir = "model_artifacts"
    os.makedirs(model_dir, exist_ok=True)
    
    # Save model in Keras format
    keras_model_path = os.path.join(model_dir, f"{model_name}.keras") 
    model.save(keras_model_path)
    
    # Save config
    config_path = os.path.join(model_dir, "config.json")
    with open(config_path, "w") as f:
        json.dump(config, f, indent=4)
    
    # Save metrics
    metrics_path = os.path.join(model_dir, "metrics.json")
    with open(metrics_path, "w") as f:
        json.dump(metrics, f, indent=4)
    
    # Create README with model info
    readme_path = os.path.join(model_dir, "README.md")
    with open(readme_path, "w") as f:
        f.write(f"# Trash Classification Model\n\n")
        f.write(f"Created on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write("## Model Architecture\n```\n")

        f.write("```\n\n")
        f.write("## Metrics\n")
        for key, value in metrics.items():
            f.write(f"- {key}: {value}\n")
    
    # Add files to artifact
    artifact.add_dir(model_dir)
    
    # Log artifact to wandb
    wandb.log_artifact(artifact)
    
    return artifact

In [13]:
model_version = "1.2"

final_metrics = {
    'final_train_loss': history.history['loss'][-1],
    'final_train_accuracy': history.history['accuracy'][-1],
    'final_val_loss': history.history['val_loss'][-1],
    'final_val_accuracy': history.history['val_accuracy'][-1],
    'total_epochs': len(history.history['loss'])
}

final_artifact = log_model_to_wandb(
    model,
    wandb.config.as_dict(),
    final_metrics,
    model_version
)

wandb.finish()

wandb: Adding directory to artifact (.\model_artifacts)... Done. 1.3s


epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
learning_rate,█████████▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▂▁▃▁▄▂▄▅▅█▅▆▆▄▆█▆▆▆▅▆▇▆█▆▇▆▇▆▇▆▆▆▇
train_loss,▇█▅▆▅█▄▄▄▂▃▂▃▃▃▂▃▂▂▃▂▁▂▁▂▂▂▂▂▂▂▂▂▁
val_accuracy,▁▂▁▁▂▃▂▂▃▃▇▇▇█▆▆▇▇██▇▇████████████
val_loss,▃▃██▃▃▃▃▃▃▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,33
learning_rate,0.0
train_accuracy,0.78125
train_loss,0.57046
val_accuracy,0.7752


In [14]:
def save_model(version, model, config, metrics, base_dir="../models"):
    model_dir = os.path.join(base_dir, f"model_{version}")
    os.makedirs(model_dir, exist_ok=True)
    
    # Save model weights and architecture
    keras_model_path = os.path.join(model_dir, f"model_{version}.keras") 
    model.save(keras_model_path)
    
    
    # Save configs
    config_path = os.path.join(model_dir, "config.json")
    with open(config_path, "w") as f:
        json.dump(config, f, indent=4)
    
    # Save metrics
    metrics_path = os.path.join(model_dir, "metrics.json")
    with open(metrics_path, "w") as f:
        json.dump(metrics, f, indent=4)
    
    # Create README
    readme_path = os.path.join(model_dir, "README.md")
    with open(readme_path, "w") as f:
        f.write(f"""---
datasets:
- garythung/trashnet
metrics:
- accuracy
pipeline_tag: image-classification
---
""")
        f.write(f"\n# Model Version {version}\n")
        f.write(f"Created on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write("```\n\n")
        f.write("## Metrics\n")
        for key, value in metrics.items():
            f.write(f"- *{key}*: {value:.4f}\n")
        
        f.write("\n## Configuration\n")
        for key, value in config.items():
            f.write(f"- **{key}**: {value}\n")
    
    print(f"Model version {version} saved successfully at {model_dir}.")
    return model_dir

In [15]:
config = {
    "architecture": "Tensorflow CNN",
    "input_size": (224, 224, 3),
    "num_classes": num_classes,
    "augmentation": {
        "rotation_range": 20,
        "width_shift_range": 0.2,
        "height_shift_range": 0.2,
        "horizontal_flip": True,
        "vertical_flip": True,
        "shear_range": 0.2,
        "brightness_range": [0.8, 1.2]
    },
    "optimizer": "adam",
    "learning_rate": 0.001,
    "batch_size": 32,
    "epochs": 50
}
    
final_metrics = {
    "train_loss": history.history['loss'][-1],
    "train_accuracy": history.history['accuracy'][-1],
    "val_loss": history.history['val_loss'][-1],
    "val_accuracy": history.history['val_accuracy'][-1]
}

save_model(model_version, model, config, final_metrics)

Model version 1.2 saved successfully at ../models\model_1.2.


'../models\\model_1.2'